In [ ]:
from ngsolve import *

from ngsolve.meshes import *
#from ngsolve.solvers import *
from ngsolve.webgui import Draw
from netgen.geom2d import unit_square

import numpy as np
#import matplotlib.pyplot as plt

#from su2_yangmills import *
#from lgt import *

In [ ]:
ne=20
#mesh = MakeStructured2DMesh(quads=True, nx=ne, ny=ne, periodic_x=True, periodic_y=True)
mesh = MakeStructured2DMesh(quads=True, nx=ne, ny=ne, periodic_x=False, periodic_y=False)
#mesh = MakeStructured2DMesh(quads=False, nx=ne, ny=ne, periodic_x=False, periodic_y=False)
#mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))
Draw(mesh)

In [ ]:
order=1
#fesC = L2(mesh, order=order)
fesC = H1(mesh, order=order)
#fesC = Discontinuous(H1(mesh, order=order, dgjumps=True))

In [ ]:
# set wind
cfwind =  CF( (y-0.5,-(x-0.5)) )

Draw(cfwind, mesh)

In [ ]:
# set charge density

def gaussxy(mu, sigma2):
    #return 1/sqrt(sigma2) * exp(-1/sigma2 *( (x-mu[0])*(x-mu[0]) + (y-mu[1])*(y-mu[1])))
    return 1/sqrt(sigma2) * gaussx(mu[0], sigma2) * gaussy(mu[1], sigma2)

def gaussx(mu, sigma2):
    return 1/sqrt(sigma2) * exp(-1/sigma2 *( (x-mu)*(x-mu)))

def gaussy(mu, sigma2):
    return 1/sqrt(sigma2) * exp(-1/sigma2 *( (y-mu)*(y-mu)))

mu= [0.5,0.3]
sigma2 = 0.01

def rho():
    #return 0.001*gaussxy(mu, sigma2) 
    return 0.001 * gaussxy(mu, sigma2) / sqrt(sqrt(sigma2))

#rhoCF = CF( (y,2*x,0) )
rhoCF = CF( (rho(), 0, 0) )

gfrho = GridFunction(fesC)

gfrho.Set( rho())

Draw(gfrho, mesh)

In [ ]:
#dt = 1.
#dt = 0.1
#dt = 0.5
dt = 0.0001

nt = 10000

In [ ]:
conv = BilinearForm(fesC, nonassemble=True)
c,cp = fesC.TnT()

conv += InnerProduct(cfwind, grad(cp))*c*dx


mc = BilinearForm(fesC)


mc += InnerProduct(c,cp)*dx


mc.Assemble()
mcmat = mc.mat
mcmatinv = mcmat.Inverse(fesC.FreeDofs())


In [ ]:


tempC = gfrho.vec.CreateVector()
newgfrho = GridFunction(fesC)

def timestep_charge(gfC, newgfC):
    newgfC.vec.data[:] = gfC.vec[:]
    conv.Apply(gfC.vec, tempC)
    newgfC.vec.data[:] += dt*mcmatinv* tempC
    gfC.vec.data[:] = newgfC.vec[:]



In [ ]:
gfCscene = Draw(gfrho, mesh)

for i in range(nt):
    timestep_charge(gfrho, newgfrho)

    
    if i % 1 == 0:
        gfCscene.Redraw()
        
    print ("\r", f"timestep:{i}", end="")
    #input()